<a href="https://colab.research.google.com/github/Cyber-Squid1/PyTorch/blob/main/CNN_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
num_epoch=4
batch_size=4
lr=0.001

In [4]:
transform=transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
    )

In [5]:
train_data=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_data=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 43290206.08it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
train_loader= torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader= torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=False)

In [7]:
classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')
num_classes=len(classes)
print(num_classes)

10


In [8]:
class ConvNet(nn.Module):
    def __init__(self,num_classes):
        super(ConvNet,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5)
        self.pool1=nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5)
        self.pool2=nn.MaxPool2d(kernel_size=2,stride=2)
        self.linear1=nn.Linear(in_features=16*5*5,out_features=128)
        self.linear2=nn.Linear(in_features=128,out_features=64)
        self.linear3=nn.Linear(in_features=64,out_features=num_classes)

    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,16*5*5) # This is used to Flatten the output from MaxPool2D
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [9]:
model=ConvNet(num_classes).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=lr)

In [10]:
n_steps=len(train_loader)
for epoch in range(num_epoch):
    for i, (images,labels) in enumerate(train_loader):
        images=images.to(device)
        labels=labels.to(device)

        output=model(images)
        loss=criterion(output,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epoch}], Step [{i+1}/{n_steps}], Loss: {loss.item():.4f}')

print("Training Finished")

Epoch [1/4], Step [2000/12500], Loss: 2.3207
Epoch [1/4], Step [4000/12500], Loss: 2.3116
Epoch [1/4], Step [6000/12500], Loss: 2.2741
Epoch [1/4], Step [8000/12500], Loss: 2.2397
Epoch [1/4], Step [10000/12500], Loss: 2.3096
Epoch [1/4], Step [12000/12500], Loss: 1.9544
Epoch [2/4], Step [2000/12500], Loss: 1.8708
Epoch [2/4], Step [4000/12500], Loss: 1.7572
Epoch [2/4], Step [6000/12500], Loss: 1.6121
Epoch [2/4], Step [8000/12500], Loss: 1.6301
Epoch [2/4], Step [10000/12500], Loss: 1.5805
Epoch [2/4], Step [12000/12500], Loss: 2.5721
Epoch [3/4], Step [2000/12500], Loss: 1.4286
Epoch [3/4], Step [4000/12500], Loss: 1.5729
Epoch [3/4], Step [6000/12500], Loss: 1.2559
Epoch [3/4], Step [8000/12500], Loss: 1.5043
Epoch [3/4], Step [10000/12500], Loss: 1.0722
Epoch [3/4], Step [12000/12500], Loss: 0.9486
Epoch [4/4], Step [2000/12500], Loss: 0.9536
Epoch [4/4], Step [4000/12500], Loss: 1.6371
Epoch [4/4], Step [6000/12500], Loss: 0.6032
Epoch [4/4], Step [8000/12500], Loss: 2.3580
Epoc

In [12]:
with torch.no_grad():
    n_correct=0
    n_samples=0
    n_class_correct=[0 for i in range(10)]
    n_class_samples=[0 for i in range(10)]
    for images,labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        output=model(images)

        _,prediction=torch.max(output,1)
        n_samples+=labels.size(0)
        n_correct+=(prediction == labels).sum().item()

        for i in range(batch_size):
            label=labels[i]
            pred=prediction[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0*n_correct/n_samples
    print(f'Accuracy of network: {acc} %')

    for i in range(10):
        acc_class=100.0*n_class_correct[i]/n_class_samples[i]
        print(f'Accuracy of class {classes[i]} is: {acc_class} %')

Accuracy of network: 46.39 %
Accuracy of class plane is: 58.1 %
Accuracy of class car is: 73.1 %
Accuracy of class bird is: 30.6 %
Accuracy of class cat is: 37.8 %
Accuracy of class deer is: 29.1 %
Accuracy of class dog is: 23.3 %
Accuracy of class frog is: 63.8 %
Accuracy of class horse is: 48.9 %
Accuracy of class ship is: 44.6 %
Accuracy of class truck is: 54.6 %
